## Google TensorFlow
- Most of notes below are my personal understanding, it may be even less accurate than other notes considering the library is released just recently.
- Google has just released its TensorFlow, which looks familiar to existing deep learning frameworks, especially theano, but with a gola of standarizing the inferface of machine learning in the world! It does it by using the same api on different devices.
- it generlizes from deep leanring framework to general machine learning tasks (model, objective and optimization)
- it wraps up api in a so far the cleanest way
- [getting started page](http://tensorflow.org/get_started)

### installation
```sh
# For CPU-only version
$ pip install https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-0.5.0-cp27-none-linux_x86_64.whl

# For GPU-enabled version (only install this version if you have the CUDA sdk installed)
$ pip install https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow-0.5.0-cp27-none-linux_x86_64.whl
```


### Some notes from tensorflow online tutorial
- Tensorflow relies on a highly efficient C++ backend to do its computation. The connection to this backend is called a `session`. The common usage for TensorFlow programs is to first create a graph and then launch it in a session.
- `InteractiveSession` allows you to interleave operations which build a computation graph with ones that run the graph. This is particularly convenient when working in interactive contexts like iPython. If you are not using an InteractiveSession, then you should build the entire computation graph before starting a session and launching the graph.
- The role of the Python code is therefore to build this external computation graph, and to dictate which parts of the computation graph should be run.
- placeholder -- a value that we'll input when we ask TensorFlow to run a computation. Its specially useful to minibatch based training. The main diff between placeholder variable and a tf.Variable is that placeholder only needs partial shape information (even that is optional, but useful for debugging), whereas a variable needs both initial value and thus shape information.
- A Variable is a value that lives in TensorFlow's computation graph. It can be used and even modified by the computation. In machine learning applications, one generally has the model paramaters be Variables.
- ***In otherwords, data are usually represented by numpy arrays. Input/output to tf graph (aka flow of graph) are presented by tf.placeholder, but models are usually represented as tf.Variable. Data can be directly used as inputs to a session graph, if the inputs don't need to change (e.g. whole batch trainig instead of minibatch)***
- Before Variables can be used within a session, they must be initialized using that session. This step takes the initial values (in this case tensors full of zeros) that have already been specified, and assigns them to each Variable. 
- When you execute the `run` of graph session, or any part of it (via a operator, e.g., train, init). you can always use `feed_dict` parameter to replace any tensor in your computation graph -- it's not restricted to just placeholders.
- Use `run` on an opeartor (node) in the graph, use `eval` on an variable in the graph, both can take `feed_dict` as parameter.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:

import tensorflow as tf
import numpy as np

In [3]:
## fitting a linear model by sgd
## data - numpy array
## variable - states of the computing graph, use session to access their values
## variable values (e.g., initial values) - tensors of tf (flows in the graph)
## session - computing graph
## drive of training - init and train operations of session (nodes in the graph)


## notice the x is the ndim x ninstances shape
x_data = np.float32(np.random.rand(2, 100))
y_data = np.dot([0.1, 0.2], x_data) + 0.3
print x_data.shape, y_data.shape

## linear model - variables with init values
## init values should be generated using tf functions, with a similiar api to numpy
W = tf.Variable(tf.random_uniform([1, 2], -1., 1.))
b = tf.Variable(tf.zeros([1]))
## like in theano, combining variable and numpy arrays to create new variable
y = tf.matmul(W, x_data) + b

## construct objective and training operations (functions)
loss = tf.reduce_mean(tf.square(y - y_data)) # variable
optimizer = tf.train.GradientDescentOptimizer(.5) # factory
train = optimizer.minimize(loss) # operation (function)

init = tf.initialize_all_variables() # operation (function)
sess = tf.Session()
sess.run(init)

for step in xrange(0, 201):
    sess.run(train)
    if step % 40 == 0:
        print step, sess.run(loss) # use session to access variables
        
print "best fit:", sess.run(W), sess.run(b)

(2, 100) (100,)
0 0.0830833
40 6.37427e-05
80 1.3638e-06
120 4.43106e-08
160 1.44728e-09
200 4.72533e-11
best fit: [[ 0.10002078  0.200018  ]] [ 0.2999807]


### Hello MNIST part I - softmax regression
- see the notes above on how tensorflow computation graph works, specially the roles of placeholder, ndarray, variables, functions and etc.
- use a Interactive session instead of a session
- design the one-layer MLP with tensorflow - so that you can interleaving building and running of the graph session.
    - use `tf.placeholder` as ***input variables*** (images and targets) to the graph for minibatch training, specifying their partial shapes. They can be thought of input flow to the graph. Compared to variables, their shapes are not fixed.
    ```python
    x = tf.placeholder("float", shape=[None, 784])
    y_ = tf.placeholder("float", shape=[None, 10])
    ```
    - create ***model variables*** using `tf.Variable` - you don't need to change their shapes during the run time, but their values will be changed during opimization. Make sure to initialize them by their intial values, all at once. `sess.run(tf.initialize_all_variables())`
    - create ***other variables***, e.g., preditected target, objective, e.g., 
    ```python
    y = tf.nn.softmax(tf.matmul(x,W) + b)
    cross_entropy = -tf.reduce_sum(y_*tf.log(y))
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    ```
    - create ***operators (computations)*** with `run` in the session, e.g., picking the optimizer, minimize the objective variable `train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)`
    - `execute the computation by either operator.run or variable.eval`

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
onehot = OneHotEncoder(n_values=10)

In [6]:
## load data
import cPickle
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = cPickle.load(open("../data/mnist.pkl"))
## TensorFlow DOESNT ACCEPT SPARSE MATRIX!
train_y = onehot.fit_transform(train_y[..., np.newaxis]).astype(np.float32).toarray()
valid_y = onehot.fit_transform(valid_y[..., np.newaxis]).astype(np.float32).toarray()
test_y = onehot.fit_transform(test_y[..., np.newaxis]).astype(np.float32).toarray()

train_x, train_y = shuffle(train_x, train_y)
print train_x.shape, valid_x.shape, test_x.shape
print train_y.shape, valid_y.shape, test_y.shape
print train_x.min(), train_x.max()
print train_x.dtype, train_y.dtype

(50000, 784) (10000, 784) (10000, 784)
(50000, 10) (10000, 10) (10000, 10)
0.0 0.996094
float32 float32


In [8]:
session = tf.InteractiveSession()
## INDEPENDENT VARIABLES - input flow to the graph
x = tf.placeholder("float32", shape = (None, 28*28))
t = tf.placeholder("float32", shape = (None, 10))

## STATE VARIBLES graph states
W = tf.Variable(tf.zeros([28*28, 10]))
b = tf.Variable(tf.zeros([10]))

## DEPENDENT VARIABLES other graph variables
y = tf.nn.softmax(tf.matmul(x, W) + b)
p = tf.arg_max(y, dimension = 1)

## objective and training method
cross_entropy = -tf.reduce_mean(tf.reduce_sum(t * tf.log(y), 1)) ## reducer, mean of sum on 2 axes
## you need cast boolean to float32 when using tf.mean, .........
classification_rate = tf.reduce_mean(tf.cast(tf.equal(p, tf.arg_max(t, dimension = 1)), "float32"))
train_fn = tf.train.GradientDescentOptimizer(0.01, ).minimize(cross_entropy)

## train loop
session.run(tf.initialize_all_variables())

batch_size = 50
for i in xrange(10000):
    ii = i % 1000
    batch_x = train_x[ii*batch_size:(ii+1)*batch_size, :]
    batch_t = train_y[ii*batch_size:(ii+1)*batch_size, :]
    #batch_x, batch_t = mnist.train.next_batch(batch_size)

    train_fn.run(feed_dict = {x: batch_x, t: batch_t}, session = session, )
    if i % 1000 == 0:
        print ("iteration %i, train_error %f" % (i, cross_entropy.eval(feed_dict = {x: batch_x, t: batch_t}))),  
        print ("valid_error %g" % cross_entropy.eval(feed_dict = {x: valid_x, t: valid_y}))
        
print "classification rate for test data: %g" % classification_rate.eval({x: test_x, t: test_y})

iteration 0, train_error 2.281228 valid_error 2.29227
iteration 1000, train_error 0.832524 valid_error 0.571073
iteration 2000, train_error 0.759068 valid_error 0.452804
iteration 3000, train_error 0.734087 valid_error 0.406345
iteration 4000, train_error 0.720655 valid_error 0.380456
iteration 5000, train_error 0.711747 valid_error 0.36354
iteration 6000, train_error 0.705170 valid_error 0.351419
iteration 7000, train_error 0.700012 valid_error 0.342197
iteration 8000, train_error 0.695814 valid_error 0.334879
iteration 9000, train_error 0.692311 valid_error 0.328889
classification rate for test data: 0.9113


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f4af4025450>> ignored


### Hello MNIST part II - CNN
- One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients.
```python
def weight_variable(shape):
  ## discard values beyond 2 * std - keep the small values
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
```
- Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons."
```python
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
```
- ***TensorFlow's 3D data flow (image data) are of the shape (ninstances, width, height, nchannels). As such, the strides and kernel sizes parameters in tf.nn.conv2d and tf.nn.max_pool all follow this shape convention. The strides will determine the final resulted convolved image size***
- ***the weight of conv layer has the shape [patchwidth, patchheight, nchannels, nfilters] for W and [nfilters] for b.***
- For example, for the vanilla of conv-maxpool, (i.e.,  convolutions uses a stride of one and are zero padded so that the output is the same size as the input. pooling is plain old max pooling over 2x2 blocks) can be implemented as 

```python
def conv2d(x, W):
  ## the filter size will be determined by w
  ## strides are all one in each dimension 
  ## - so the convolved image has the same size with the original one
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  ## 2x2 kernels, with 2x2 stride (without overlapping)
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
```
- the cnn implemented below has the structure (conv, maxpool, conv, maxpool, dense, dropout, softmax)

In [9]:
## load the cifar data
import cPickle
from sklearn.preprocessing import OneHotEncoder
def load_cifar_train():
    imgpath = "../data/cifar-10-batches-py/"
    image_batchs, label_batchs = [], []
    for f in ["data_batch_%i" for i in range(1, 6)]:
        batch = cPickle.load(open("../data/cifar-10-batches-py/data_batch_1"))
        image_batchs.append(np.array([np.transpose(r.reshape((3, 32, 32)), (1, 2, 0)) for r in batch["data"]]))
        label_batchs.append(np.array(batch["labels"]))
    images = np.concatenate(image_batchs, axis = 0)
    labels = np.concatenate(label_batchs, axis = 0)
    return (images, labels)

def load_cifar_test():
    batch = cPickle.load(open("../data/cifar-10-batches-py/test_batch"))
    images = np.array([np.transpose(r.reshape((3, 32, 32)), (1, 2, 0)) for r in batch["data"]])
    labels = np.array(batch["labels"])
    return (images, labels)

cifar_train_images, cifar_train_labels = load_cifar_train()
cifar_test_images, cifar_test_labels = load_cifar_test()

cifar_train_images = np.multiply(cifar_train_images, 1./255)
cifar_test_images = np.multiply(cifar_test_images, 1./255)
onehot = OneHotEncoder(10)
cifar_train_labels = onehot.fit_transform(cifar_train_labels[..., np.newaxis]).toarray()
cifar_test_labels = onehot.fit_transform(cifar_test_labels[..., np.newaxis]).toarray()

cifar_train_images = cifar_train_images.astype(np.float32)
cifar_train_labels = cifar_train_labels.astype(np.float32)
cifar_test_images = cifar_test_images.astype(np.float32)
cifar_test_labels = cifar_test_labels.astype(np.float32)

print cifar_train_images.shape, cifar_train_labels.shape
print cifar_test_images.shape, cifar_test_labels.shape
print cifar_train_images.min(), cifar_train_images.max()
print cifar_train_images.dtype, cifar_train_labels.dtype

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)
0.0 1.0
float32 float32


In [10]:
from time import time

In [15]:
## build the network

## helper function on creating model variable with proper init values
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def bias_variable(shape):
    return tf.Variable(tf.constant(value = 0.1, shape = shape))

## input/output place holders
x_image = tf.placeholder(dtype = np.float32, shape = (None, 32, 32, 3))
y_label = tf.placeholder(dtype = np.float32, shape = (None, 10))

## layer1 conv/maxpool - with relu layer
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])
## x_image : (None, 32, 32, 3), W_conv1 : (5, 5, 3, 32), generate (?, ?, 3, 32)
## strides with all 1s will result in the same image size
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=(1,1,1,1), padding="SAME") 
                     + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize = (1, 2, 2, 1), 
                         strides = (1, 2, 2, 1), padding="SAME")

## layer2 conv/maxpool - with relu layer - double filter sizes
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=(1, 1, 1, 1), padding="SAME")
                    + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize = (1, 2, 2, 1), strides=(1, 2, 2, 1), 
                        padding = "SAME")

## fully connected dense layer 
## - now the image flow should have the size (None, 8, 8, 64), with 1024 hidden dimension
W_fc1 = weight_variable([8*8*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_reshaped = tf.reshape(h_pool2, (-1, 8*8*64))
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_reshaped, W_fc1) + b_fc1)

## now we need regularization - dropout layer
## need to control dropout as it is on when training and generally off when predicting
## again use a placeholder as a controller
keep_prob = tf.placeholder(np.float32) # it is a scalar
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, )

## softmax layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_fc2 = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## cost function and predicted class labels
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y_label * tf.log(y_fc2), 1))
predicted_label = tf.arg_max(y_fc2, dimension=1)
match = tf.equal(tf.arg_max(y_label, dimension=1), predicted_label)
classification_rate = tf.reduce_mean(tf.cast(match, np.float32))

## training function
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## training loop
sess = tf.Session()
sess.run(tf.initialize_all_variables())
batch_size = 128
for epoch in xrange(50000):
    i = epoch % (cifar_train_images.shape[0] / batch_size)
    batch = slice(i*batch_size, (i+1)*batch_size)
    x_data, y_data = cifar_train_images[batch, ...], cifar_train_labels[batch, ...]
    tic = time()
    train_step.run(feed_dict={x_image: x_data, y_label: y_data, keep_prob : .5}, 
                   session = sess)
    
    if epoch % 100 == 0:
        toc = time() - tic
        print epoch, "train error", cross_entropy.eval(feed_dict={x_image:x_data, 
                                                          y_label:y_data, 
                                                          keep_prob:1.}, session = sess), 
        print 'time (s)', toc
#     if epoch % 1000 == 0:
#         print "test error", cross_entropy.eval(feed_dict={x_image:cifar_test_images[:1000,...], 
#                                                           y_label:cifar_test_labels[:1000, ...], 
#                                                           keep_prob:1.}, session = sess)
        
        
print "test classification rate: ", classification_rate.eval(feed_dict={x_image:cifar_test_images, 
                                                          y_label:cifar_test_labels, 
                                                        keep_prob:1.}, session = sess)

0 train error 8.16461 time (s) 0.409779787064
100 train error 1.83488 time (s) 0.371289968491
200 train error 1.59829 time (s) 0.443302154541
300 train error 1.74269 time (s) 0.395637989044
400 train error 1.51177 time (s) 0.415384054184
500 train error 1.52278 time (s) 0.420225858688
600 train error 1.42883 time (s) 0.431298971176
700 train error 1.40683 time (s) 0.369115114212
800 train error 1.33825 time (s) 0.399115085602
900 train error 1.34097 time (s) 0.392896175385
1000 train error 1.20071 time (s) 0.37681221962
1100 train error 1.04692 time (s) 0.380440950394
1200 train error 1.20273 time (s) 0.340277910233
1300 train error 1.136 time (s) 0.429105043411
1400 train error 1.00207 time (s) 0.39781498909
1500 train error 0.962834 time (s) 0.387616157532
1600 train error 1.08313 time (s) 0.378707885742
1700 train error 1.00179 time (s) 0.3853328228
1800 train error 0.890499 time (s) 0.385201931
1900 train error 0.768781 time (s) 0.390058040619
2000 train error 0.829846 time (s) 0.4

KeyboardInterrupt: 

In [16]:
classification_rate.eval(feed_dict={x_image:cifar_test_images, 
                                                          y_label:cifar_test_labels, 
                                                                        keep_prob:1.}, session = sess)

0.59240001